[ rewrote server in tornado /mytornado/mytornado.py ]

[ trying to run flask with sockit io ]

[ now running with web page closed ]

[ now running with gunicorn ]

[ now i reduced complexity of bTrial getState]

[here i decreased javascript update from 400 ms to 5000 ms]

[BINGO!!!! This reduces errors even more!!!!]

Here we ran a Teensy generating frames every 30 ms for 1 minute and we will analyze the resultant frames the Raspberry received.

Conclusions:

 - The Pi missed 50 out of 1998 frames
 - The frame-arrival errors get worse the later into the trial.


## Load trial file directly from PiE server

In [11]:
import os
import pandas as pd
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go

init_notebook_mode(connected=True)

import urllib2 # python 2
from StringIO import StringIO # python 2

# Read the trial file from a url on the local PiE server
"""
url = 'http://192.168.1.15:5010/videolist/home/pi/video/20180805/20180805_190456_t1.txt'
response = urllib2.urlopen(url).read()
html = StringIO(response)
"""

html = '/Volumes/pi15/video/20180806/20180806_233603_t3.txt'
df = pd.read_csv(html, header=1)

frame_df = df.loc[df['event'].isin(['frame'])]

frame_df = df[ df['event'].isin(['frame']) ]
#frame_df = frame_df[ frame_df['lastFrameInterval'] != '0']

print 'Number of frames generated by Teensy running (60 seconds) simScope = 1998'
print 'Number of frames generated by Teensy running (30 seconds) simScope = 998'
print 'Number of frames PiE server running on Raspberry detected = ', frame_df.shape[0]

frame_df[0:9]


Number of frames generated by Teensy running (60 seconds) simScope = 1998
Number of frames generated by Teensy running (30 seconds) simScope = 998
Number of frames PiE server running on Raspberry detected =  1998


,date,time,linuxSeconds,secondsSinceStart,event,value,str
4,20180806,23:36:03,1.533613e+09,0.069908,frame,1,NaN
5,20180806,23:36:03,1.533613e+09,0.098815,frame,2,NaN
6,20180806,23:36:03,1.533613e+09,0.128614,frame,3,NaN
7,20180806,23:36:03,1.533613e+09,0.158491,frame,4,NaN
8,20180806,23:36:03,1.533613e+09,0.189183,frame,5,NaN
9,20180806,23:36:03,1.533613e+09,0.218905,frame,6,NaN
10,20180806,23:36:03,1.533613e+09,0.248701,frame,7,NaN
11,20180806,23:36:03,1.533613e+09,0.278262,frame,8,NaN
12,20180806,23:36:03,1.533613e+09,0.309026,frame,9,NaN


## Plot the frame time versus frame number.

Looks pretty good, right?

In [12]:
fig = {
    'data': [{
        'x': frame_df.value, 
        'y': frame_df.secondsSinceStart, 
        'text': '', 
        'mode': 'markers', 
        'name': ''
    }],
    'layout': {
        'xaxis': {'title': 'Frame Number'},
        'yaxis': {'title': "Seconds Since Start (sec)"}
    }
}

iplot(fig, filename='pandas/multiple-scatter')

## Plot the previous frame interval for each frame

It seems the errors in frame intervals gets progressively worse?

In [13]:
myFrameDiff = np.diff( frame_df['secondsSinceStart'].astype(float).values )
#myFrameDiff = np.diff( frame_df['ticks'].astype(float).values ) / 1000.0
#myFrameDiff = np.diff( frame_df['perf_counter'].astype(float).values ) 
#myFrameDiff = frame_df['lastFrameInterval'].astype(float).values / 1000

fig = {
    'data': [{
        'x': frame_df.value, 
        'y': myFrameDiff, 
        'text': '', 
        'mode': 'markers', 
        'name': ''
    }],
    'layout': {
        'xaxis': {'title': 'Frame Number'},
        'yaxis': {'title': "Inter-Frame-Interval (seconds)"}
    }
}

iplot(fig, filename='pandas/multiple-scatter')

In [14]:
histData = go.Histogram(x=myFrameDiff)

fig = {
    'data': [histData],
    'layout': {
        'xaxis': {'title': 'Inter-Frame-Interval (seconds)'},
        'yaxis': {'title': "Count", 'type': "linear"}
    }
}

iplot(fig, filename='pandas/multiple-scatter')

## Switching the y-axis from linear to log exposes all our errors

In [15]:
histData = go.Histogram(x=myFrameDiff)

fig = {
    'data': [histData],
    'layout': {
        'xaxis': {'title': 'Inter-Frame-Interval (seconds)'},
        'yaxis': {'title': "Count", 'type': "log"}
    }
}

iplot(fig, filename='pandas/multiple-scatter')

## Shot plot of frame interval [i+1] versus frame interval [i]

## The histogram of frame intervals doesn't look so bad.

Now look at each frame interval versus the previous frame interval. In general, this is called a shot plot. The negative correlation here tells us long frame intervals (y-axis) are preceded by short frame intervals (x-axis). Conversly, short frame intervals (y-axis) are preceded by long frame intervals (x-axis). One interpretation of this is there is no correlation between the delays in successive intervals. A long interval, which is an error, is followed by a short interval whose frame is actually on time but the time since the last frame is shorter (because the previous frame was late). Put another way, this might tell us the errors in frame times are random. 

In [16]:
print 'number of frame intervals = ', myFrameDiff.shape[0]

# this assumes we have an even number
interval0 = myFrameDiff[0::2]
interval1 = myFrameDiff[1::2]

fig = {
    'data': [{
        'x': interval0, 
        'y': interval1, 
        'text': '', 
        'mode': 'markers', 
        'name': ''
    }],
    'layout': {
        'xaxis': {'title': 'Previous Inter-Frame-Interval'},
        'yaxis': {'title': "Inter-Frame-Interval (seconds)"}
    }
}

iplot(fig, filename='pandas/multiple-scatter')

number of frame intervals =  1997
